<a href="https://colab.research.google.com/github/zhukuixi/Pytorch_tutorial/blob/main/softmax_mlp.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import torch
import pandas as pd
import numpy as np

In [2]:
!pip install git+https://github.com/d2l-ai/d2l-zh@release  # installing d2l
!pip install matplotlib_inline
import torch
from IPython import display
from d2l import torch as d2l
import numpy as np



Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Cloning https://github.com/d2l-ai/d2l-zh (to revision release) to /tmp/pip-req-build-bfn3qyrl
  Running command git clone --filter=blob:none --quiet https://github.com/d2l-ai/d2l-zh /tmp/pip-req-build-bfn3qyrl
  Running command git checkout -b release --track origin/release
  Switched to a new branch 'release'
  Branch 'release' set up to track remote branch 'release' from 'origin'.
  Resolved https://github.com/d2l-ai/d2l-zh to commit b29df2a66b5569984330b936409045eb2c13ef29
  Running command git submodule update --init --recursive -q
  Preparing metadata (setup.py) ... done
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


/usr/local/lib/python3.9/dist-packages/torch/cuda/__init__.py:497: UserWarning: Can't initialize NVML
  warnings.warn("Can't initialize NVML")


In [3]:
from torch import nn

In [4]:
# metric - accuracy
def accuracy(y_hat,y):
  #“==”opearation is sensitive to dype 
  # we need to do the dtype transformation before == operation
  if len(y_hat)>1 and y_hat.shape[1]>1:
      y_hat = y_hat.argmax(axis=1)
  return (y_hat.type(y.dtype)==y).sum()


In [ ]:
class logisticNet(torch.nn.Module):
  def __init__(self):
    super().__init__()
    self.flatten = torch.nn.Flatten()
    self.linear = torch.nn.Linear(784,10)
  def forward(self,x):
    return torch.sigmoid(self.linear(self.flatten(x)))

class MLP(torch.nn.Module):
  def __init__(self):
    super().__init__()
    self.net = nn.Sequential(nn.Flatten(),
                    nn.Linear(784, 256),
                    nn.ReLU(),
                    nn.Linear(256, 10))
  def forward(self,x):
    return self.net(x)

train_iter,test_iter = d2l.load_data_fashion_mnist(50)
#net = logisticNet()
net = MLP()
lossFun = torch.nn.CrossEntropyLoss(reduction='none')
optimizier = torch.optim.SGD(net.parameters(),lr=0.1)

for epoch in np.arange(100):
  for X,y in train_iter:
    optimizier.zero_grad()
    y_hat = net(X)
    loss = lossFun(y_hat, y)  
    loss.mean().backward()
    optimizier.step()
 
  print("epoch:{} loss:{:.3}".format(epoch,loss.mean().item()))

  with torch.no_grad():
    acc = 0      
    count = 0 
    for X_test,y_test in test_iter:
      acc += accuracy(net(X_test),y_test)
      count += y_test.numel()
    print("test accuracy:{:.3}".format(acc/count))
    

